# Simple Neural Networks

First, we create a simpler CNN architecture. 

# Convolutional Neural Network

In [ ]:
# TODO: Create Data Generator 


In [ ]:
# TODO: Create and Train Model 


In [ ]:
# TODO: Create Predictions


In [ ]:
# TODO: Visualize Results

## Artificial Neural Network

The other model is an Artificial Neural Network with five hidden layers consisting of eight neurons each, with a tansigmoid transfer function. 

In [ ]:
# TODO: Create Data Generator 


In [ ]:
# TODO: Create and Train Model 


In [ ]:
# TODO: Create Predictions


In [ ]:
# TODO: Visualize Results